# test report: swath start

The algorithm to detect the first sample that can safely be used for
swath processing depends on a number of assumptions and heuristics. With
that, it is one of the most susceptible parts of the algorithm.

I assume that in a regular case, the range window start above the
surface with basically no signal present. The first sample that
intersects the surface is the point of closest approach POCA. However,
in detail the "first sample" is not easy to define. In mountainous
regions, this first observable intersection is ahead of or behind what
we usually view as "footprint". From there, the signal builds up
(leading edge), until the power and the coherence assume a high value.
This is where I define the POCA: at the first coherence peak. Again,
what one considers the first peak involves choices. I use a rather early
point (see the code for details) which might lead to a positive bias
(which depends on the along-track slope -- the steeper, the bigger the
overestimation). After that, the area that contributes to the signal
disperses and ultimately splits into a left and right side around the
POCA. With that the coherence drops, until the left and right sides are
far enough apart (~10 km) and their signals do not interfere anymore.
After this point, or when one of the sides starts to dominate the
signal, we can use the remaining samples to retrieve swath elevation
estimates. Mind that range ambiguities can appear, also, at any later
location; which is why one usually requires a minimum coherence for any
processed sample.

With those assumptions and choices, I search for the maximum of the
smoothed coherence in a 10 m window after the coherence first exceeds
the coherence threshold and name this sample the POCA. In another window
from 5 to 50 m after the POCA, I search for a rise in coherence. This is
where the swath processing starts.

In the following, "edge cases" are recognized by an early or late swath
start. For the _k_ earliest and latest of those, the coherence waveform
is plotted for visual inspection. Also for another _k_ random ones.

In [ ]:
k_zero = 3
k_smallest = 3
k_biggest = 3
k_random = 3

In [ ]:
import os, sys
sys.path.insert(0, os.path.join("..", ".."))

In [ ]:
from cryoswath import l1b

In [ ]:
l1b_data = l1b.from_id("20140517T225409").append_best_fit_phase_index()

In [ ]:
import numpy as np

In [ ]:
import random

In [ ]:
import cryoswath.test_plots.waveform

In [ ]:
def plot(idx):
    cryoswath.test_plots.waveform.coherence(l1b_data.isel(time_20_ku=[idx]))
    cryoswath.test_plots.waveform.power(l1b_data.isel(time_20_ku=[idx]))
    cryoswath.test_plots.waveform.dem_transect(l1b_data.isel(time_20_ku=[idx]))

In [ ]:
[plot(idx) for idx in random.sample(np.argwhere(l1b_data.swath_start.values==0).flatten().tolist(), k=k_zero)];

In [ ]:
sortkeys = np.argsort(l1b_data.swath_start.values)[sum(l1b_data.swath_start.values==0):]

In [ ]:
[plot(idx) for idx in sortkeys[:k_smallest]];

In [ ]:
[plot(idx) for idx in sortkeys[-k_biggest:]];

In [ ]:
[plot(idx) for idx in random.sample(sortkeys.tolist(), k=k_random)];